# Fabio Junco Amaral
## Harvard Extensions School, Fall 2018 
***
***

# We will now start investigating some data from Boston 

# We will do a lot of data wrangling based on the CSV file found at (https://data.boston.gov/dataset/food-establishment-inspections)

## Lets start importing some libraries that will help us

In [ ]:
# Libraries
import pandas as pd # For data manipulation and analysis
import matplotlib.pyplot as plt # Better plots
from itertools import groupby

# Using the 'inline', my matplotlib graphs will be included in my notebook, next to the code
% matplotlib inline


## We will now read the file, and rename some columns, so we can have a common code (and following PEP8)

In [ ]:
df = pd.read_csv('mayorsfoodcourt.csv', encoding = "ISO-8859-1", low_memory=False)

df_food = df.rename(index=str, columns={'LICENSENO': 'license_num', 'LICSTATUS': 'license_st', 'DESCRIPT': 'biz_ype', 
                             'ViolLevel': 'violation_lvl', 'ViolDesc': 'violation_desc', 'ZIP': 'zip', 
                              'VIOLDTTM': 'violation_date', 'ViolStatus': 'violation_st'})


In [ ]:
# Lets look our dataframe now
# The tail() shows by default the last five rows
df_food.tail(2)

## There is too much information in this dataframe, lets get rid of some columns that we will not use

In [ ]:
df_food = df_food.drop(['businessName', 'DBAName', 'LegalOwner', 'NameLast', 'RESULT', 'RESULTDTTM', 'NameFirst',
              'LICENSECAT', 'Comments', 'STATE', 'Property_ID', 'Violation', 'StatusDate', 
              'Location', 'ISSDTTM', 'EXPDTTM', 'Address', 'CITY'], axis=1)

## Now we have More data wrangling to do

In [ ]:
df_food['zip'] = df_food['zip'].astype(str)  # We need the zipcode to be a string, to easily manipulate its values
df_food = df_food[df_food['zip'].map(len) >= 4]  # There are many invalid zips, we will map only the 4 last digits
df_food['zip'] = df_food['zip'].astype(int)  # Turning the zip into ints again
df_food = df_food[df_food['violation_st'] != ' ']  # We will remove whitespaces from violation_st
df_food = df_food[df_food['violation_lvl'] != ' ']  # We will remove whitespaces from violation_lvl
df_food = df_food[df_food['license_st'] != 'Deleted']  # We will remove invalid values from license_st

## Most of the work when we are dealing with data is cleaning so we can better use it. We can clean and modified as much as we want to. Here is a quote about data wrangling:

### "Data wrangling is the process of gathering, selecting, and transforming data to answer an analytical question.  Also known as data cleaning or “munging”, legend has it that this wrangling costs analytics professionals as much as 80% of their time, leaving only 20% for exploration and modeling. Why does it take so long to wrangle the data so that it is usable for analytics?"
https://www.elderresearch.com/blog/what-is-data-wrangling

## After all that, we will filter for business located in Boston.
### We found the valid Boston Zip codes in this [website](http://www.city-data.com/zipmaps/Boston-Massachusetts.html)

In [ ]:
boston_zip = ['2108', '2109', '2110', '2111', '2113', '2114', '2115', '2116', '2118', '2119', '2120', '2121', 
              '2122', '2124', '2125', '2126', '2127', '2128', '2129', '2130', '2131', '2132', '2133', '2134', 
              '2135', '2136', '2163', '2199', '2203', '2210', '2215', '2222']

df_food = df_food[df_food.zip.isin(boston_zip)]

## Alright, now lets start creating some plots

### First, how many violations have Passed or Fail (keep in mind that each business have multiple violations)

In [ ]:
# We create that will count all values from violation_st creating an index
ind = df_food['violation_st'].value_counts().index
# Now we will plot this column, counting the values
df_food['violation_st'].value_counts()[ind].plot(kind='bar', figsize=(6, 6));

# A good chart must have labels telling you what it is showing to you
plt.xlabel('Violation Status')
plt.ylabel('Number of Violations')
plt.title('Number of Violation per Violation Status');

## It looks like we have more Fails than Passes

### Let's check the level of severity of these violations

In [ ]:
ind = df_food['violation_lvl'].value_counts().index
df_food['violation_lvl'].value_counts()[ind].plot(kind='bar', figsize=(6, 6));

plt.xlabel('Violation Level')
plt.ylabel('Number of Violations')
plt.title('Number of Violation per Violation Level');

## Lets check more about the Violations that Failed

In [ ]:
# We can create a new dataframe, only with the violation_st that is 'Fail'
df_fail = df_food[df_food['violation_st'] == 'Fail']

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
explode = (0, 0.1, 0) # only "explode" the 2nd slice
labels = '* = Low', '*** = Severe', '** = Medium'  # It is ordered, so we have to put the labels in order too

ind = df_fail['violation_lvl'].value_counts().index
df_fail['violation_lvl'].value_counts()[ind].plot(kind='pie', figsize=(10, 10), labels=labels, explode=explode, 
                                                  autopct='%1.1f%%', shadow=True,  startangle=90);

plt.xlabel('')
plt.ylabel('')
plt.title('Percentage of Violation Level by Failed Violations');

In [ ]:
# Let's quickly see the same graph for violation_st that is 'Pass'
df_pass = df_food[df_food['violation_st'] == 'Pass']

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
explode = (0, 0.1, 0) # only "explode" the 2nd slice
labels = '* = Low', '*** = Severe', '** = Medium'  # It is ordered, so we have to put the labels in order too

ind = df_pass['violation_lvl'].value_counts().index
df_pass['violation_lvl'].value_counts()[ind].plot(kind='pie', figsize=(10, 10), labels=labels, explode=explode, 
                                                  autopct='%1.1f%%', shadow=True,  startangle=90);

plt.xlabel('')
plt.ylabel('')
plt.title('Percentage of Violation Level by Failed Violations');

### So almost 20% of the violations that failed are severe. Lets dig more into that.

In [ ]:
# We will create a new dataframe (based on the dataset with on fails), only violation_lvl equal '*** (severe)
df_fail_severe = df_fail[df_fail['violation_lvl'] == '***']

# The head() shows by default the first five rows, but lets check only the last 2 rows
df_fail_severe.head(2)

## Where in the city of Boston are those violations happening? We will check the Fails that are Severe

In [ ]:
ind = df_fail_severe['zip'].value_counts().index
df_fail_severe['zip'].value_counts()[ind].plot(kind='bar', figsize=(12, 12));

plt.xlabel('Boston Zip Code')
plt.ylabel('Severe Violations Fail')
plt.title('Severe Violations Fail by Boston Zip Code');

## The Zip Code that has more fails (more than severe 4000 violations) is in the Zip Code 2116.
### If we look for this zip, we find that this is the Back Bay area. Maybe because there is a lot of restaurants and business in that area, a lot of old houses? We can try to conclude WHY we have such a high number of severe cases there.

### What if we dig even deeper into this?

In [ ]:
# New dataframe, only with zipcode 2116
df_fail_severe_2116 = df_fail_severe[df_fail_severe['zip'] == 2116]

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
explode = (0, 0.1) # only "explode" the 2nd slice

ind = df_fail_severe_2116['license_st'].value_counts().index
df_fail_severe_2116['license_st'].value_counts()[ind].plot(kind='pie', figsize=(10, 10), explode=explode, 
                                                  autopct='%1.1f%%', shadow=True,  startangle=90);

plt.xlabel('')
plt.ylabel('')
plt.title('Percentage of Violation Level by Failed Violations');

## So it looks like almost 70% of those businesses are still Actives


## This is just an example of what we can do with a dataset. We can gather pieces of information and conclusions based on data.
## I personally find Graphs amazing, they can tell so much just by looking by then and trying to get what they are telling us.

## Pandas, Numpy and Matplotlib are amazing, and they can offer MUCH more than I've used here. I will now give some random examples of cool stuff we can do with them.

In [ ]:
# Combining different data in one graph

# We can manipulate the size of our graph with this call
plt.figure(figsize=(10, 7))

# Plotting two graphs
# It's possible to modify the kind of the plot (pie chart, bar, line etc), width, color, labels and MUCH more
df_fail.groupby('license_st').violation_lvl.value_counts().plot(kind='bar', color='r', label='FAIL', 
                                                                zorder=10, linewidth=5.0)
df_pass.groupby('license_st').violation_lvl.value_counts().plot(kind='line', color='g', label='PASS', 
                                                                zorder=15, linewidth=5.0)


# Creating a label box outside the chart
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel('License Status')
plt.ylabel('Count')
plt.title('Combination of Active and Inactive License status with Violation Severity');

In [ ]:
# Graph custozimation is almost endless. We will modify this graph as much as we can (not too worried with information
# displayed, we will get crazy)

# Manipulating ploting parameter separeted
plt.figure(figsize=(10, 10))
plt.rcParams['text.color'] = 'black'
plt.rcParams['font.size'] = 12
plt.rcParams['font.weight'] = 'bold'
colors_x = ['#2AAF33', '#002B00', '#004600']
colors_y = ['#FF4D3A', '#CB0707', '#FF1800']
labels = '* = Low', '*** = Severe', '** = Medium'  # It is ordered, so we have to put the labels in order too

indx = df_fail['violation_lvl'].value_counts().index
x = df_fail['violation_lvl'].value_counts()[indx]

indy = df_pass['violation_lvl'].value_counts().index
y = df_pass['violation_lvl'].value_counts()[indy]
 
# Plot
plt.pie(y, colors=colors_y, autopct='%1.1f%%', pctdistance=0.85, startangle=180, labeldistance=1.08, radius=1,
        labels=labels)
plt.pie(x, colors=colors_x, autopct='%1.5f%%', pctdistance=0.85, startangle=180, radius=0.75, frame=True)

# Creating a doughnut style chart
centre_circle = plt.Circle((0,0),0.5,color='black', fc='white',linewidth=0)
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
 
plt.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 15))

# A scatter plot is a type of plot that shows the data as a collection of points. The position of a point depends 
# on its two-dimensional value, where each value is a position on either the horizontal or vertical dimension. 
plt.scatter(df_fail['zip'], df_fail['license_num'], c="purple", alpha=0.5, marker='x', label="X")

plt.xlabel("Number")
plt.ylabel("Zip")
plt.legend(loc='upper left')
plt.show()

In [ ]:
# ORDER MATTERS!!!

# The same code as before, but we will change then posisiton of some lines

# plt.figure(figsize=(15, 15)) <- This code will go below

plt.scatter(df_fail['zip'], df_fail['license_num'], c="purple", alpha=0.5, marker='x', label="X")

plt.figure(figsize=(5, 5))  # <- here it is

plt.xlabel("Number")
plt.ylabel("Zip")
plt.legend(loc='upper left')
plt.show()

# We also can do everything we can with python in this dataframe, anything to help debug, wrangling data and whatever our imagination let us 

In [ ]:
for columns in df_fail_severe_2116:
    print(columns, type(columns), len(columns))

# Lets talk a little about Markdowns 
> ## <font color=green>They are one of the most important parts of our notebook</font>
***
### We need to be as clear as possible in our graphs and explanations
#### So why not use words to do it?

- Customize
- Be __bold__
    - But no _too_ much

***
### Now, before we finish, I would like to point out some limitations of the project and how to address them in the future. The main issue was the SIZE of the dataframe. It had almost 550 thousand rows and 26 columns. This is a very big dataframe.
### Even after I've done some data cleaning, I'd still had a big dataframe.
### So this was a problem because some graphs I was trying to do took so much time that I decided to not put them into this project. I believe that I need to use NumPy in the future because NumPy can greatly improve my performance issue.

# Thank you for your time!